In [2]:
import os, re, sys, utilities
spark_home = os.environ.get('SPARK_HOME', None)

In [3]:
code = utilities.getSparkContext()
exec(code)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2.3.1.4.26-3
      /_/

Using Python version 3.6.8 (default, Dec 30 2018 01:22:34)
SparkSession available as 'spark'.


In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, HiveContext

In [5]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import datetime
import math
import scipy

In [6]:
from pyspark.sql.functions import col, udf, countDistinct, isnan, when, count, to_date, from_unixtime, unix_timestamp
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import Row

In [7]:
from pyspark.sql.types import DoubleType, FloatType, BooleanType, LongType, \
IntegerType, TimestampType, StringType, StructType, StructField, ArrayType

### Read in parquet file from HDFS - preprocessed EU data

In [8]:
# using SQLContext to read parquet file
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(SparkContext)

# to read parquet file - EU Preprocessed
df = spark.read.parquet('hdfs://hdp2cluster/user/lguerra5/pae_processing_prod/trip_summary_preprocessed_subset_EU/*')

In [9]:
df.show(10)

+-------------+------------------------+-----------------+-----------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|trip_end_date|trip_end_eng_oil_percent|trip_end_odo_read|              vin|trip_number|oil_change_flag|oil_change_cycle_number|first_ol_pct|first_odometer|first_ol_date|
+-------------+------------------------+-----------------+-----------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|   2019-07-01|                   100.0|               26|6FPPXXMJ2PKE89768|          0|              0|                      0|       100.0|         19784|   2019-11-21|
|   2019-07-02|                   100.0|               26|6FPPXXMJ2PKE89768|          1|              0|                      0|       100.0|         19784|   2019-11-21|
|   2019-07-02|                   100.0|               44|6FPPXXMJ2PKE89768|          2|              0|                      0|       100.0|    

In [13]:
df.printSchema()

root
 |-- trip_end_date: date (nullable = true)
 |-- trip_end_eng_oil_percent: float (nullable = true)
 |-- trip_end_odo_read: long (nullable = true)
 |-- vin: string (nullable = true)
 |-- trip_number: integer (nullable = true)
 |-- oil_change_flag: integer (nullable = true)
 |-- oil_change_cycle_number: integer (nullable = true)
 |-- first_ol_pct: float (nullable = true)
 |-- first_odometer: long (nullable = true)
 |-- first_ol_date: date (nullable = true)



### Create Test Data for SE team

In [22]:
#vehicle line description 
df.select("vin").distinct().show()

+-----------------+
|              vin|
+-----------------+
|6FPPXXMJ2PKS33137|
|6FPPXXMJ2PKS33767|
|WF01XXERK1JR51276|
|WF01XXERK1KD07553|
|WF01XXERK1KJ74862|
|WF01XXERK1KK89994|
|WF01XXERK1KL30849|
|WF01XXERK1KL33839|
|WF01XXERK1KS67232|
|WF0EXXTTREKJ43517|
|WF0EXXTTREKS04947|
|WF0EXXTTREKS26013|
|WF0FXXGAHFKE49489|
|WF0JXXGAHJJA35459|
|WF0JXXGAHJKC65979|
|WF0JXXGAHJKD29216|
|WF0JXXGAHJKK08863|
|WF0JXXGAHJKK86086|
|WF0JXXGAHJKL72420|
|WF0JXXGAHJKL73592|
+-----------------+
only showing top 20 rows



In [10]:
#get rid of nulls and unknowns in vin column
#Fl_crashes_tbl = Fl_crashes_tbl.filter(~Fl_crashes_tbl.vin.isin(["","UK","UNKNOWN","UNKOWN","UKNOWN","UNK","UNKN","UN","UKN","NONE","0","NA","00","1"]))

## Save one VIN as CSV for E2E

In [23]:
del one_vin

In [24]:
#check for empty values of vin field (all empty values are just empty string and not null/nan)
one_vin = df.filter((df["vin"] == "6FPPXXMJ2PKE89768"))
#no further clean up needed!

In [25]:
one_vin.show(5)

+-------------+------------------------+-----------------+-----------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|trip_end_date|trip_end_eng_oil_percent|trip_end_odo_read|              vin|trip_number|oil_change_flag|oil_change_cycle_number|first_ol_pct|first_odometer|first_ol_date|
+-------------+------------------------+-----------------+-----------------+-----------+---------------+-----------------------+------------+--------------+-------------+
|   2019-07-01|                   100.0|               26|6FPPXXMJ2PKE89768|          0|              0|                      0|       100.0|         19784|   2019-11-21|
|   2019-07-02|                   100.0|               26|6FPPXXMJ2PKE89768|          1|              0|                      0|       100.0|         19784|   2019-11-21|
|   2019-07-02|                   100.0|               44|6FPPXXMJ2PKE89768|          2|              0|                      0|       100.0|    

In [29]:
# Preprocessed Data
print(one_vin.agg({"trip_number": "max"}).collect()[0][0])
print(one_vin.agg({"trip_number": "min"}).collect()[0][0])

1034
0


In [ ]:
# Preprocessed Data
print(df.agg({"trip_end_odo_read": "max"}).collect()[0][0])
print(df.agg({"trip_end_odo_read": "min"}).collect()[0][0])

In [28]:
test_data = one_vin.toPandas()

In [30]:
test_data.to_csv("/s/lguerra5/eu_test_data_01.csv")

### Read Trip Summary Table (filtered using query)

In [ ]:
# filtered trip summary data, original
trip_sum_NA_raw = spark.sql("""SELECT T1.cvdcqa_vin_d_3 AS vin,
coalesce(T1.cvdcqa_oil_chng_cyc_no_r_3,0) AS oil_change_cycle_number,
T1.cvdcqa_trip_no_r_3 AS trip_number,
to_date(T1.cvdcqa_trip_end_time_s_3) AS trip_end_date,
T1.cvdcqa_trip_end_eng_oil_p_3 AS trip_end_eng_oil_percent,
T1.cvdcqa_trip_end_odo_read_r_3 AS trip_end_odo_read
FROM cvdp.NCVDCQA_TRIP_SUM_FINAL_SEC_HTE T1
WHERE T1.cvdcqa_suspect_flag_r_3 = 0 AND
T1.cvdcqa_trip_no_r_3 IS NOT NULL AND
T1.cvdcqa_trip_end_time_s_3 IS NOT NULL AND
T1.cvdcqa_trip_end_eng_oil_p_3 IS NOT NULL AND
T1.cvdcqa_trip_end_odo_read_r_3 IS NOT NULL AND
T1.cvdcqa_vin_d_3 IS NOT NULL AND
T1.cvdcqa_partition_cntry_x = 'USA' AND
year(T1.cvdcqa_trip_end_time_s_3) > 2015""")

In [8]:
# filtered trip summary data
trip_sum_EU_filtered = spark.sql("""SELECT T1.cvdcqa_vin_d_3 AS vin,
T1.cvdcqa_trip_end_eng_oil_p_3 as trip_end_eng_oil_percent,
to_date(T1.cvdcqa_trip_end_time_s_3) AS trip_end_date,
T1.cvdcqa_vehicle_description888_x_3 AS veh_dsc,
T1.cvdcqa_mvemodyr7 AS model_year
FROM cvdp.NCVDCQA_TRIP_SUM_FINAL_SEC_HTE T1
WHERE T1.cvdcqa_suspect_flag_r_3 = 0 AND
T1.cvdcqa_trip_no_r_3 IS NOT NULL AND
T1.cvdcqa_trip_end_time_s_3 IS NOT NULL AND
T1.cvdcqa_trip_end_odo_read_r_3 IS NOT NULL AND
T1.cvdcqa_vin_d_3 IS NOT NULL AND
T1.cvdcqa_partition_region_x = 'EU' AND
year(T1.cvdcqa_trip_end_time_s_3) > 2015""")

In [1]:
trip_sum_EU_filtered.printSchema()
#trip_sum_NA_raw.printSchema()

NameError: name 'trip_sum_EU_filtered' is not defined

In [ ]:
trip_sum_EU_filtered.show(10)

### Raw trip Sum Data  - check how many suspect trips

In [ ]:
# filtered trip summary data
trip_sum_EU_raw = spark.sql("""SELECT T1.cvdcqa_vin_d_3 AS vin,
coalesce(T1.cvdcqa_oil_chng_cyc_no_r_3,0) AS oil_change_cycle_number,
T1.cvdcqa_trip_no_r_3 AS trip_number,
to_date(T1.cvdcqa_trip_end_time_s_3) AS trip_end_date,
T1.cvdcqa_trip_end_eng_oil_p_3 AS trip_end_eng_oil_percent,
T1.cvdcqa_trip_end_odo_read_r_3 AS trip_end_odo_read, T1.cvdcqa_suspect_flag_r_3
FROM cvdp.NCVDCQA_TRIP_SUM_FINAL_SEC_HTE T1
WHERE T1.cvdcqa_trip_no_r_3 IS NOT NULL AND
T1.cvdcqa_trip_end_time_s_3 IS NOT NULL AND
T1.cvdcqa_trip_end_eng_oil_p_3 IS NOT NULL AND
T1.cvdcqa_trip_end_odo_read_r_3 IS NOT NULL AND
T1.cvdcqa_vin_d_3 IS NOT NULL AND
T1.cvdcqa_partition_region_x = 'EU' AND
year(T1.cvdcqa_trip_end_time_s_3) > 2015""")

In [ ]:
trip_sum_EU_raw.filter((trip_sum_EU_raw["cvdcqa_suspect_flag_r_3"] != "0")).count()

### Function to ID Suspect Trips (based on weekly metrics script)

In [ ]:
def oilDropReasonableDueToOdo(oil_drop_series,odo_increase_series):
    if (oil_drop_series <  odo_increase_series/8000.0*100.0*1.5 + 3).any():
        return True
    else:
        return False
    
def oilDropReasonableDueToTime(oil_drop_series,time_delta_series):
    if (oil_drop_series <  time_delta_series/86400.0/180*100*1.5 + 3).any():
        return True
    else:
        return False
    
def oilDropReasonable(oil_drop_series,odo_increase_series,time_delta_series):
    if oilDropReasonableDueToOdo(oil_drop_series,odo_increase_series) \
    or oilDropReasonableDueToTime(oil_drop_series, time_delta_series):
        return False
    else:
        return True  

In [ ]:
def SuspectOilLife(df):
#     oilLifeChanged = df.oil.diff()
#     oilDrop = df.oil.diff() < 0
    oilBigDrop = df.oil.diff() < -1
    oilJump = df.oil.diff() > 0
    suspectOilDropOccurs = False
    suspectOilJumpOccurs = False
    oilJumpOccurs = False
    if oilBigDrop.any() == True:
        oil_drop_series = -df.oil.diff()[oilBigDrop]
        odo_increase_series = df.odo.diff()[oilBigDrop]
        time_delta_series = df.time_s.diff()[oilBigDrop].apply(func=lambda td:td.total_seconds())
        suspectOilDropOccurs = oilDropReasonable(oil_drop_series,odo_increase_series,time_delta_series)
    if oilJump.any() == True:
        oilJumpOccurs = True
        oil_jump_series = 100-df.oil[oilJump]
        odo_increase_series = df.odo.diff()[oilJump]
        time_delta_series = df.time_s.diff()[oilJump].apply(func=lambda td:td.total_seconds())
        suspectOilJumpOccurs = oilDropReasonable(oil_jump_series,odo_increase_series,time_delta_series)
        
    return suspectOilDropOccurs,suspectOilJumpOccurs,oilJumpOccurs

In [ ]:
columnNames = trip_sum_EU_filtered.schema.names

def Combined(_row):    #Turns value a (a tuple) into a list of a single tuple.
    return [_row]

def MergeValue(_list, _row): #a is the new type [(,), (,), ..., (,)] and b is the old type (,)
    _list.append(_row)
    return _list

def MergeCombiners(_list1, _list2): #a is the new type [(,),...,(,)] and so is b, combine them
    _longlist = _list1 + _list2
    return _longlist

def Rename(dataf):
    dataf.columns = columnNames
    return dataf

def Reindex(dataf):
    dataf.index = range(0,len(dataf))
    return dataf

In [ ]:
oilLifeTripSumPdRdd = trip_sum_EU_filtered.rdd.keyBy(lambda row:row.vin)\
.combineByKey(Combined,MergeValue,MergeCombiners,numPartitions=3000)\
.map(lambda tuple: pd.DataFrame(tuple[1])).map(Rename).map(lambda dataf: dataf.sort_values(['odo','time_s']))\
.map(Reindex)

### Questions to answer:

#### 1. How many trips were ignored by pre-processing?

In [ ]:
#count number of trips (rows) for EU preprocessed data
df.select("vin").count()

In [ ]:
#count number of trips (rows) for raw Trip Sum data
#trip_sum_NA_raw.select("vin").count()

In [ ]:
#count number of trips (rows) for EU Trip Sum data
trip_sum_EU_filtered.select("vin").count()

In [ ]:
# ratio of messages rejected EU
(140867273-92163566)/float(140867273)

In [ ]:
#check for empty values of vin field (all empty values are just empty string and not null/nan)
#adas.filter((adas["vin2"] == "") | adas["vin2"].isNull() | isnan(adas["vin2"])).count()
#no further clean up needed!

#### 2. How many uique VINs

In [ ]:
df.select("vin").distinct().count()

In [ ]:
trip_sum_EU_filtered.select("vin").distinct().count()

#### 3. Date range

In [ ]:
# Preprocessed Data
max_val1 = df.agg({"trip_end_date": "max"}).collect()[0][0]
min_val1 = df.agg({"trip_end_date": "min"}).collect()[0][0]

In [ ]:
print(max_val1)
print(min_val1)

In [ ]:
# Raw Trip Summay data
max_val2 = trip_sum_EU_filtered.agg({"trip_end_date": "max"}).collect()[0][0]
min_val2 = trip_sum_EU_filtered.agg({"trip_end_date": "min"}).collect()[0][0]

In [ ]:
print(max_val2)
print(min_val2)

#### 4. Suspect Trips

### Ratio of suspected data (based on weekly metrics script)

In [ ]:
suspectedCount = oilLifeTripSumPdRdd.map(SuspectOilLife).countByValue()
suspectedCount

In [ ]:
d_week = dict(suspectedCount)
float(d_week[(False, True, True)] + d_week[(True, False, True)] + d_week[(True, True, True)]) / \
(d_week[(False, False, False)] + d_week[(False, False, True)])

In [ ]:
def oilResetOil(df):
    oilJump = df.oil.diff() > 0
    return df.oil.shift(1)[oilJump].as_matrix()

In [ ]:
x = oilLifeTripSumPdRdd\
.filter(lambda df:SuspectOilLife(df)==(False, False, True))\
.flatMap(oilResetOil)\
.countByValue()

In [ ]:
dictx = dict(x)
oilLifeList = []
countList = []
for k in dictx:
    oilLifeList.append(k)
    countList.append(dictx[k])
    
oilResetPd = pd.DataFrame({'oil':oilLifeList,'count':countList})

In [ ]:
oilResetPd.to_csv("/s/lguerra5/oilReset"+START_DATE_1.replace("-","",2)+"_"+END_DATE_1.replace("-","",2)+".csv")

#### 5. List of Deleted Trips

#### 6. Deep Dive